In [3]:
import numpy as np
import pandas as pd
import re
import glob
import os 
from sklearn.model_selection import GridSearchCV,cross_val_score,cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

In [4]:
path_train="../window/w_scores/w_stack_train/"
path_test="../window/w_scores/w_stack_test/"

In [5]:
dict_train = {} 
dict_test = {}
train_files = [i for i in os.listdir(path_train)]
test_files = [i for i in os.listdir(path_test)]
for f in train_files:
    dict_train[re.sub("\.csv","",f)] = pd.read_csv(path_train+f)
for f in test_files:
    dict_test[re.sub("\.csv","",f)] = pd.read_csv(path_test+f)

In [6]:
col_no_ID=['class1', 'class2', 'class3', 'class4', 'class5', 'class6', 'class7',
       'class8', 'class9']

In [7]:
#Affecting  columns name
for name in dict_test.keys():
    dict_train[name].columns=[name+"_"+s for s in col_no_ID]+["ID"]
    dict_test[name].columns=[name+"_"+s for s in col_no_ID]+["ID"]

In [8]:
stack_train=dict_train["w_xgb_d2v"].merge(dict_train["w_xgb_tfidf"],
on="ID").merge(dict_train["w_xgb_w2v"],on="ID").merge(dict_train["w_lgbm_d2v"],
on="ID").merge(dict_train["w_lgbm_tfidf"],on="ID").merge(dict_train["w_lgbm_w2v"],
on="ID").merge(dict_train["w_lr_d2v"],on="ID").merge(dict_train["w_lr_tfidf"],
on="ID").merge(dict_train["w_lr_w2v"],on="ID").merge(dict_train["w_ada_d2v"],
on="ID").merge(dict_train["w_ada_tfidf"],on="ID").merge(dict_train["w_ada_w2v"],
on="ID").merge(dict_train["w_rf_d2v"],on="ID").merge(dict_train["w_rf_tfidf"],
on="ID").merge(dict_train["w_rf_w2v"],on="ID")
#a tester 

In [9]:
stack_test=dict_test["w_xgb_d2v"].merge(dict_test["w_xgb_tfidf"],
on="ID").merge(dict_test["w_xgb_w2v"],on="ID").merge(dict_test["w_lgbm_d2v"],
on="ID").merge(dict_test["w_lgbm_tfidf"],on="ID").merge(dict_test["w_lgbm_w2v"],
on="ID").merge(dict_test["w_lr_d2v"],on="ID").merge(dict_test["w_lr_tfidf"],
on="ID").merge(dict_test["w_lr_w2v"],on="ID").merge(dict_test["w_ada_d2v"],
on="ID").merge(dict_test["w_ada_tfidf"],on="ID").merge(dict_test["w_ada_w2v"],
on="ID").merge(dict_test["w_rf_d2v"],on="ID").merge(dict_test["w_rf_tfidf"],
on="ID").merge(dict_test["w_rf_w2v"],on="ID")

In [14]:
#quand le merging est fini : table unique, on peut monter un meta_model

In [11]:
y=pd.read_csv("../bases/new_training_variants.csv").Class.values - 1

In [12]:
ID_test=stack_test["ID"]
ID_train=stack_train["ID"]

In [13]:
meta_feat_train=pd.read_csv("../window/w_meta_features/meta_train_l1l2.csv")
meta_feat_test=pd.read_csv("../window/w_meta_features/meta_test_l1l2.csv")

In [14]:
magic_feat=pd.read_csv("../l2_meta_features/svd25_molecular_functions.csv")
magic_feat=magic_feat.drop(["Class","Gene","ID","Variation"],axis=1)
magic_train=magic_feat[:len(y)]
magic_test=magic_feat[len(y):]
magic_train=magic_train.reset_index(drop=True)
magic_test=magic_test.reset_index(drop=True)

In [15]:
stack_train_1=pd.concat((stack_train,meta_feat_train,magic_train),axis=1)
stack_test_1=pd.concat((stack_test,meta_feat_test,magic_test),axis=1)

In [16]:
meta_feat_train=meta_feat_train.drop("Unnamed: 0",axis=1)
meta_feat_test=meta_feat_test.drop("Unnamed: 0",axis=1)

In [17]:
X_train=np.array(stack_train_1.drop("ID",axis=1))

In [18]:
X_test=np.array(stack_test_1.drop("ID",axis=1))

In [20]:
kf = StratifiedKFold(n_splits=5, random_state=26, shuffle=True)

In [26]:
#Ridge vs Lasso searching
param_test= {
    "C" : [0.1,0.3,0.5,0.7,0.9,1,2,3,4,5],
    "penalty": ["l1","l2"]
}


In [27]:
gsearch=GridSearchCV(estimator=LogisticRegression(), param_grid = param_test,scoring="neg_log_loss",n_jobs=4,iid=False, cv=kf)
gsearch.fit(X_train,y)
gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_


C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.92355, std: 0.03656, params: {'C': 0.1, 'penalty': 'l1'},
  mean: -0.90141, std: 0.03732, params: {'C': 0.1, 'penalty': 'l2'},
  mean: -0.88315, std: 0.03961, params: {'C': 0.3, 'penalty': 'l1'},
  mean: -0.88530, std: 0.03827, params: {'C': 0.3, 'penalty': 'l2'},
  mean: -0.87617, std: 0.04131, params: {'C': 0.5, 'penalty': 'l1'},
  mean: -0.88300, std: 0.04029, params: {'C': 0.5, 'penalty': 'l2'},
  mean: -0.87322, std: 0.04229, params: {'C': 0.7, 'penalty': 'l1'},
  mean: -0.88458, std: 0.04131, params: {'C': 0.7, 'penalty': 'l2'},
  mean: -0.87436, std: 0.04314, params: {'C': 0.9, 'penalty': 'l1'},
  mean: -0.88372, std: 0.04049, params: {'C': 0.9, 'penalty': 'l2'},
  mean: -0.87496, std: 0.04380, params: {'C': 1, 'penalty': 'l1'},
  mean: -0.88218, std: 0.04254, params: {'C': 1, 'penalty': 'l2'},
  mean: -0.88438, std: 0.04582, params: {'C': 2, 'penalty': 'l1'},
  mean: -0.88286, std: 0.04145, params: {'C': 2, 'penalty': 'l2'},
  mean: -0.88819, std: 0.04593, params: {'

In [28]:
#model with parameters found on gridsearch

In [32]:
def stacker_gen(X,X_test,y,classifier):
    kf = StratifiedKFold(n_splits=5, random_state=26, shuffle=True)
    fold = 0
    y_test=0
    for train_index, test_index in kf.split(X, y):
        
        fold += 1

        X_train, X_valid    = X[train_index],   X[test_index]
        y_train, y_valid    = y[train_index],   y[test_index]

        print("Fold", fold, X_train.shape, X_valid.shape)

        clf=classifier
        clf.fit(X_train,y_train)
        p_test = clf.predict_proba(X_test)
        y_test += p_test/5
    classes = "class1,class2,class3,class4,class5,class6,class7,class8,class9".split(',')
    subm = pd.DataFrame(y_test, columns=classes)
    subm['ID'] = ID_test
    
    subm.to_csv("../final_scores/w_stack_test.csv",index=False)
    
    print("cross_val sur train ") #peut etre que to array est exclusivement pour les xgb
    

    y_pred=cross_val_predict(estimator=clf,X=X,y=y,cv=kf,method="predict_proba")
    subm1 = pd.DataFrame(y_pred, columns=classes)
    subm1['ID'] = ID_train
    subm1.to_csv("../final_scores/w_stack_train.csv",index=False)


In [33]:
#parametres à rentrer

In [34]:
clf = LogisticRegression(C=0.7,penalty="l1")
stacker_gen(X_train,X_test,y,clf)


Fold 1 (2946, 203) (743, 203)
Fold 2 (2949, 203) (740, 203)
Fold 3 (2952, 203) (737, 203)
Fold 4 (2954, 203) (735, 203)
Fold 5 (2955, 203) (734, 203)
cross_val sur train 
